# Notebook 1: MED-PC Extracting the Recording Data and Metadata

# Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os
from datetime import datetime

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from medpc2excel.medpc_read import medpc_read
from moviepy.editor import *

In [3]:
# setting path
sys.path.append('../../../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

# Getting the Metadata from all the MED-PC Recording Files

- Every MED-PC Recording file has Metadata about time, data, subject, group, experiment name, script used, and the MED-PC box number on the top of the recording file. We will first extract that information
    - Metadata can be background information or context about data(or files) that is often different from the actual data that is intended to be recorded.  
        For more information on Metadata: https://en.wikipedia.org/wiki/Metadata

In [5]:
cohort = "pilot_2"

In [6]:
# Getting all the file paths of the recording files(that happen to all end in `.txt`)
all_med_pc_file = glob.glob("./data/*.txt")

- The path of the directory that this notebook is in. This will be where relative paths will be based off of

In [7]:
current_working_directory = os.getcwd()

In [8]:
current_working_directory

'/root/projects/behavioral_dataframe_processing/results/2022_10_12_pilot_1_2_3_reward_training/pilot_2'

- All the other files in this directory. If you want to use a folder in here such as the `data` folder, you'd type `./data/{name_of_folder}` where you replace `{name_of_folder}` with the name of the folder without the `{}`. You will do this in the cells following the one below
    - The `./` means the path will reference the current directory that the command is being used from

In [9]:
os.listdir(current_working_directory)

['data',
 '03_calculating_port_entry_precision.ipynb',
 '.ipynb_checkpoints',
 'old',
 'proc',
 '02_calculating_latencies.ipynb',
 '01_extracting_recording_data_and_metadata.ipynb']

## NOTE: If you are using your own data, the following path must be changed to the directory where your MED-PC recording files are located(if they are not in the specied folder). It is recommended to create a sub folder in the `data` folder(that is in the same folder as this notebook) for each group of session you want to process and put the recording files in there

- Use the cell below instead if you're using your own data. You must change the cell below by clicking on the cell and then pressing the `esc` key. Then press the `Y` button on your keyboard. If you want to switch it back do the same but press the `R` button instead. This switches it from a cell that is run as if it's code, or back to a cell where it's just treated as text
    - **NOTE: If you are using a different folder, then change the path in the cell below**
    - The asterisk is called a wild card which will tell the computer to find all files/folders that match this pattern. The `*` means that any pattern of any length can be replaced with it. The `**` tells the computer to look in all folders that are in the specified folder. For more information: https://linuxhint.com/bash_wildcard_tutorial/
    - For more information on finding the path of your folder that contains the recording files: https://www.computerhope.com/issues/ch001708.htm

In [10]:
all_med_pc_file = glob.glob("./data/*.txt")

In [11]:
all_med_pc_file[:10]

['./data/2022-07-22_12h26m_Subject 1.txt',
 './data/2022-07-23_14h55m_Subject 4.txt',
 './data/2022-07-23_10h06m_Subject 2.txt',
 './data/2022-07-27_15h10m_Subject 3.txt',
 './data/2022-07-26_11h36m_Subject 4.txt',
 './data/2022-07-22_10h11m_Subject 2.txt',
 './data/2022-07-22_09h00m_Subject 3.txt',
 './data/2022-07-26_09h05m_Subject 3.txt',
 './data/2022-07-22_11h19m_Subject 3.txt',
 './data/2022-07-23_14h55m_Subject 3.txt']

# Filtering the MED-PC Files that have Pilot of Pilot in it

- Example of what the MED-PC Recording file looks like

In [12]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-07-22_12h26m_Subject 1.txt







Start Date: 07/22/22

End Date: 07/22/22

Subject: 1

Experiment: C57_vs_CD1_comparison 

Group: Cage4

Box: 1

Start Time: 12:26:57

End Time: 13:36:44

MSN: CD1_reward_training

A:    4399.000

D:    9000.000

E:       0.000

L:       0.000

M:       0.000

O:       0.000

T:    3660.000



- We will be extracting the first 10 or so lines that look like:

```
File: C:\MED-PC\Data\2022-05-06_12h59m_Subject 3.4 (2).txt

Start Date: 05/06/22

End Date: 05/06/22

Subject: 3.4 (2)

Experiment: Pilot of Pilot

Group: Cage 4

Box: 1

Start Time: 12:59:58

End Time: 14:02:38

MSN: levelNP_CS_reward_laserepochON1st_noshock
```

- We will just find all the lines that start with `"File", "Start Date", "End Date", "Subject", "Experiment", "Group", "Box", "Start Time", "End Time", or "MSN"`. And get the metadata from those lines. And then stop once all the metadata types have been collected

In [13]:
# This makes a nested dictionary of file paths to each individual metadata type
file_path_to_meta_data = extract.metadata.get_all_med_pc_meta_data_from_files(list_of_files=all_med_pc_file)

In [14]:
# The metadata for the first file
for key, value in file_path_to_meta_data.items():
    print("File path: {}".format(key))
    print("Metadata types and associated values: {}".format(value))
    break

File path: ./data/2022-07-22_12h26m_Subject 1.txt
Metadata types and associated values: {'File': 'C:\\MED-PC\\Data\\2022-07-22_12h26m_Subject 1.txt', 'Start Date': '07/22/22', 'End Date': '07/22/22', 'Subject': '1', 'Experiment': 'C57_vs_CD1_comparison', 'Group': 'Cage4', 'Box': '1', 'Start Time': '12:26:57', 'End Time': '13:36:44', 'MSN': 'CD1_reward_training'}


## Making a Dataframe out of the Metadata

- A Dataframe is essentially a "programmable" spreadsheet. But instead of clicking on cells, you will have to tell Python how you want to interact with the spreadsheet
    - For more information: https://realpython.com/pandas-dataframe/

In [15]:
# Turning the dictionary into a Pandas Dataframe
metadata_df = pd.DataFrame.from_dict(file_path_to_meta_data, orient="index")
# Resetting the index because currently the file path is the index 
metadata_df = metadata_df.reset_index()

In [16]:
metadata_df.head()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
0,./data/2022-07-22_12h26m_Subject 1.txt,C:\MED-PC\Data\2022-07-22_12h26m_Subject 1.txt,07/22/22,07/22/22,1,C57_vs_CD1_comparison,Cage4,1,12:26:57,13:36:44,CD1_reward_training
1,./data/2022-07-23_14h55m_Subject 4.txt,C:\MED-PC\Data\2022-07-23_14h55m_Subject 4.txt,07/23/22,07/23/22,4,C57_vs_CD1_Comparison,Cage 6,1,14:55:47,16:05:03,CD1_reward_training
2,./data/2022-07-23_10h06m_Subject 2.txt,C:\MED-PC\Data\2022-07-23_10h06m_Subject 2.txt,07/23/22,07/23/22,2,C57_vs_CD1_comparison,Cage2,3,10:06:22,11:09:48,CD1_reward_training
3,./data/2022-07-27_15h10m_Subject 3.txt,C:\MED-PC\Data\2022-07-27_15h10m_Subject 3.txt,07/27/22,07/27/22,3,C57_vs_CD1_Comparison,Cage 6,4,15:10:22,16:16:08,CD1_reward_training
4,./data/2022-07-26_11h36m_Subject 4.txt,C:\MED-PC\Data\2022-07-26_11h36m_Subject 4.txt,07/26/22,07/26/22,4,C57_vs_CD1_Comparison,Cage3,4,11:36:00,12:41:44,C57_reward_training


In [17]:
metadata_df.tail()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
235,./data/2022-07-27_12h52m_Subject 4.txt,C:\MED-PC\Data\2022-07-27_12h52m_Subject 4.txt,07/27/22,07/27/22,4,C57_vs_CD1_comparison,Cage 4,1,12:52:20,13:55:50,CD1_reward_training
236,./data/2022-07-26_11h36m_Subject 2.txt,C:\MED-PC\Data\2022-07-26_11h36m_Subject 2.txt,07/26/22,07/26/22,2,C57_vs_CD1_Comparison,Cage3,2,11:36:00,12:41:44,C57_reward_training
237,./data/2022-07-20_10h48m_Subject 3.txt,C:\MED-PC\Data\2022-07-20_10h48m_Subject 3.txt,07/20/22,07/20/22,3,C57_vs_CD1_Comparison,Cage3,1,10:48:32,11:53:40,C57_reward_training
238,./data/2022-07-25_14h43m_Subject 2.txt,C:\MED-PC\Data\2022-07-25_14h43m_Subject 2.txt,07/25/22,07/25/22,2,C57_vs_CD1_Comparison,cage6,1,14:43:50,15:46:38,CD1_reward_training
239,./data/2022-07-24_14h53m_Subject 3.txt,C:\MED-PC\Data\2022-07-24_14h53m_Subject 3.txt,07/24/22,07/24/22,3,C57_vs_CD1_comparison,Cage 6,1,14:53:31,15:56:03,CD1_reward_training


- Getting the numbers out of the column that contains the cage information

In [18]:
import re

In [19]:
metadata_df = metadata_df.dropna(subset="Group")

In [20]:
metadata_df = metadata_df[metadata_df["Group"] != ""]

In [21]:
metadata_df["cage"] = metadata_df["Group"].apply(lambda x: re.findall(r'\d+', x)[0])

In [22]:
metadata_df["cage"].unique()

array(['4', '6', '2', '3', '1', '5'], dtype=object)

In [23]:
metadata_df["cage"].head()

0    4
1    6
2    2
3    6
4    3
Name: cage, dtype: object

- Getting the number of files that are associated with each subject

In [24]:
# How many files there are for each subject
metadata_df.groupby("Subject").count()

,index,File,Start Date,End Date,Experiment,Group,Box,Start Time,End Time,MSN,cage
Subject,,,,,,,,,,,
1,60,60,60,60,60,60,60,60,60,60,60
2,60,60,60,60,60,60,60,60,60,60,60
3,60,60,60,60,60,60,60,60,60,60,60
4,60,60,60,60,60,60,60,60,60,60,60


## Looking over the MED-PC scripts

- MED-PC has scripts(lines of code that act as the instructions for how to operate the MED-PC boxes) that contain the descriptions of what each value in the recordings are. We will extract the descriptions from the MED-PC scripts so that we can label the data points in the MED-PC recordings  
- Below is a list of all the MED-PC Scripts. We will only be looking at the first one    

In [25]:
all_medpc_scripts = glob.glob("./data/*.MPC")

In [26]:
all_medpc_scripts

['./data/C57_reward_competition.MPC',
 './data/CD1_reward_training.MPC',
 './data/CD1_reward_competition.MPC',
 './data/CD1_reward.MPC',
 './data/C57_reward_training.MPC',
 './data/C57_reward.MPC']

- Example of what the MED-PC Script looks like that was ran when recording the behaviors

In [27]:
with open(all_medpc_scripts[0]) as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line)

\v3 stop tone with poke

\v3.2 monitor port entries AND exits



\INPUTS

^port = 8



\OUTPUTS

^fan = 16

^houselight = 11

^tone1 = 2

^tone2 = 3

^tone3 = 4

^tone4 = 5

^pump = 9

^whitenoise = 1

^csout = 5

^peout = 15

^cs1out = 17

^cs2out = 13

^cs3out = 14



\EXP SETTINGS

^ncsNoShock = 0

^initCS1trials = 3



\ARRAYS

DIM P = 20000 \Port entry time stamp array

DIM Q = 2500 \US delivery time stamp array (absolute)

DIM R = 2500 \US time stamp array (relative to last CS)

DIM W = 2500 \ITI values used for CS

DIM S = 2500 \CS presentation values (absolute - every time light turns on)

DIM N = 20000 \Port exit time stamp array

DIM K = 2500 \CS type

DIM G = 2500 \controlled_stimulus_seconds computer clock time (seconds on clock every time light turns on)

DIM H = 2500 \controlled_stimulus_minutes computer clock time (minutes on clock every time light turns on)

DIM I = 2500 \controlled_stimulus_hours computer clock time (hours every time light turns on)

DIM B = 2500 \port

- We will be using the comments in the MED-PC script(Everything after the `\` for each line) to create a name for the variables. By default, MED-PC uses a single letter as the name of the variable(programming object that holds some information).
    - This will use the medpc2excel library found in https://github.com/cyf203/medpc2excel
- Example of the variable names and their comments in the MED-PC script that we will get the descriptive names from:
    - MED-PC will squish the first few words to create the name for every variable

```
DIM P = 20000 \Port entry time stamp array

DIM Q = 2500 \US delivery time stamp array (absolute)

DIM R = 2500 \US time stamp array (relative to last CS)

DIM W = 2500 \ITI values used for CS

DIM S = 2500 \CS presentation values (absolute - every time light turns on)

DIM N = 20000 \Port exit time stamp array

DIM K = 2500 \CS type

DIM B = 2500 \shock intensity
```

- In the MED-PC recording files, there are values that are labelled with letters. These data type of these letters is described in the MED-PC script file that we just looked at.

In [28]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-07-22_12h26m_Subject 1.txt







Start Date: 07/22/22

End Date: 07/22/22

Subject: 1

Experiment: C57_vs_CD1_comparison 

Group: Cage4

Box: 1

Start Time: 12:26:57

End Time: 13:36:44

MSN: CD1_reward_training

A:    4399.000

D:    9000.000

E:       0.000

L:       0.000

M:       0.000

O:       0.000

T:    3660.000



## **NOTE: Please make sure that the corresponding `.mpc` file (aka the MED-PC script) that was ran to create the log file, is also in the same folder as the recording files. This notebook will fail to extract the data from the recording file if it is missing**

- Spreadsheet of the data from the MED-PC recording file for all the files combined into one.
- **NOTE: Each row does not correspond to the same data point. Each row represents the "n"-th data point for each category for each file. AKA, the first row is the first data point for the time the subject entered the port and the first data point for the time that the tone that is played, two seperate things. The second row is the second data point for all the categories, the third row is the third data point and so on. This repeats for all data points in a given file, then it starts over to the next file where the first row of that file is the first for all its data points.**

In [29]:
concatted_medpc_df = extract.dataframe.get_medpc_dataframe_from_list_of_files(medpc_files=all_med_pc_file)

In [30]:
concatted_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,6.22,64.0,399.0,0.0,60.01,6.26,1.0,44.0,36.0,12.0,50.0,35.0,12.0,20220722,1,./data/2022-07-22_12h26m_Subject 1.txt
1,6.28,144.0,399.0,0.0,140.01,6.36,1.0,4.0,38.0,12.0,50.0,35.0,12.0,20220722,1,./data/2022-07-22_12h26m_Subject 1.txt
2,6.38,234.0,399.0,0.0,230.01,6.41,1.0,34.0,39.0,12.0,50.0,35.0,12.0,20220722,1,./data/2022-07-22_12h26m_Subject 1.txt
3,6.44,314.0,399.0,0.0,310.01,6.52,1.0,54.0,40.0,12.0,51.0,35.0,12.0,20220722,1,./data/2022-07-22_12h26m_Subject 1.txt
4,6.64,389.0,399.0,0.0,385.01,6.69,1.0,9.0,42.0,12.0,51.0,35.0,12.0,20220722,1,./data/2022-07-22_12h26m_Subject 1.txt


In [31]:
concatted_medpc_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
2805,3651.72,NaN,NaN,NaN,NaN,3651.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/2022-07-24_14h53m_Subject 3.txt
2806,3651.81,NaN,NaN,NaN,NaN,3651.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/2022-07-24_14h53m_Subject 3.txt
2807,3655.18,NaN,NaN,NaN,NaN,3655.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/2022-07-24_14h53m_Subject 3.txt
2808,3655.31,NaN,NaN,NaN,NaN,3655.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/2022-07-24_14h53m_Subject 3.txt
2809,3655.42,NaN,NaN,NaN,NaN,3655.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/2022-07-24_14h53m_Subject 3.txt


In [32]:
pilot_df = concatted_medpc_df[~concatted_medpc_df["subject"].str.contains(r'v', na=True)].copy()

In [33]:
pilot_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,6.22,64.0,399.0,0.0,60.01,6.26,1.0,44.0,36.0,12.0,50.0,35.0,12.0,20220722,1,./data/2022-07-22_12h26m_Subject 1.txt
1,6.28,144.0,399.0,0.0,140.01,6.36,1.0,4.0,38.0,12.0,50.0,35.0,12.0,20220722,1,./data/2022-07-22_12h26m_Subject 1.txt
2,6.38,234.0,399.0,0.0,230.01,6.41,1.0,34.0,39.0,12.0,50.0,35.0,12.0,20220722,1,./data/2022-07-22_12h26m_Subject 1.txt
3,6.44,314.0,399.0,0.0,310.01,6.52,1.0,54.0,40.0,12.0,51.0,35.0,12.0,20220722,1,./data/2022-07-22_12h26m_Subject 1.txt
4,6.64,389.0,399.0,0.0,385.01,6.69,1.0,9.0,42.0,12.0,51.0,35.0,12.0,20220722,1,./data/2022-07-22_12h26m_Subject 1.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2805,3651.72,NaN,NaN,NaN,NaN,3651.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/2022-07-24_14h53m_Subject 3.txt
2806,3651.81,NaN,NaN,NaN,NaN,3651.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/2022-07-24_14h53m_Subject 3.txt
2807,3655.18,NaN,NaN,NaN,NaN,3655.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/2022-07-24_14h53m_Subject 3.txt
2808,3655.31,NaN,NaN,NaN,NaN,3655.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220724,3,./data/2022-07-24_14h53m_Subject 3.txt


In [34]:
pilot_df["date"].unique()

array(['20220722', '20220723', '20220727', '20220726', '20220719',
       '20220718', '20220720', '20220721', '20220724', '20220725'],
      dtype=object)

In [35]:
concatted_medpc_df = pilot_df.copy()

- Combining the recording and the metadata into one dataframe

In [36]:
recording_and_metadata_df = concatted_medpc_df.merge(metadata_df, left_on='file_path', right_on='index')

In [37]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage
0,6.22,64.0,399.0,0.0,60.01,6.26,1.0,44.0,36.0,12.0,...,07/22/22,07/22/22,1,C57_vs_CD1_comparison,Cage4,1,12:26:57,13:36:44,CD1_reward_training,4
1,6.28,144.0,399.0,0.0,140.01,6.36,1.0,4.0,38.0,12.0,...,07/22/22,07/22/22,1,C57_vs_CD1_comparison,Cage4,1,12:26:57,13:36:44,CD1_reward_training,4
2,6.38,234.0,399.0,0.0,230.01,6.41,1.0,34.0,39.0,12.0,...,07/22/22,07/22/22,1,C57_vs_CD1_comparison,Cage4,1,12:26:57,13:36:44,CD1_reward_training,4
3,6.44,314.0,399.0,0.0,310.01,6.52,1.0,54.0,40.0,12.0,...,07/22/22,07/22/22,1,C57_vs_CD1_comparison,Cage4,1,12:26:57,13:36:44,CD1_reward_training,4
4,6.64,389.0,399.0,0.0,385.01,6.69,1.0,9.0,42.0,12.0,...,07/22/22,07/22/22,1,C57_vs_CD1_comparison,Cage4,1,12:26:57,13:36:44,CD1_reward_training,4


## Getting the cage numbers and the dates so that we can include it in the file name

- We will get the metadata from the recording files and use it to name the files we will create. This will help organize the files and make it easy to know where each file came from 

- Getting the group numbers(original cage names)

In [38]:
# removing blank spaces
group_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["Group"].unique() if number]
# sorting numbers
group_numbers = sorted(group_numbers)
group_numbers_for_title = "_".join(group_numbers)

In [39]:
group_numbers_for_title

'Cage1_Cage2_Cage3_Cage4_Cage5_Cage6_Cage_1_Cage_2_Cage_3_Cage_4_Cage_5_Cage_6_cage5_cage6'

- Getting the cage numbers

In [40]:
# removing blank spaces
cage_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["cage"].unique() if number]
# sorting numbers
cage_numbers = sorted(cage_numbers)
cage_numbers_for_title = "_".join(cage_numbers)

In [41]:
cage_numbers_for_title

'1_2_3_4_5_6'

- Getting the dates

In [42]:
# Getting the first and last recording date to get a range
earliest_date = recording_and_metadata_df["date"].min()
latest_date = recording_and_metadata_df["date"].max()

In [43]:
earliest_date

'20220718'

In [44]:
latest_date

'20220727'

- Getting the subject names

In [45]:
recording_and_metadata_df["Subject"].unique()

array(['1', '4', '2', '3'], dtype=object)

In [46]:
# removing blank spaces
subject_numbers = ["subject-" + number for number in recording_and_metadata_df["Subject"].unique() if number]
# sorting numbers
subject_numbers = sorted(subject_numbers)
subject_numbers_for_title = "_".join(subject_numbers)

In [47]:
subject_numbers_for_title

'subject-1_subject-2_subject-3_subject-4'

- Getting the experiment name

In [48]:
experiment_names = ["_".join(name.split()) for name in recording_and_metadata_df["Experiment"].unique() if name]
# sorting experiment_names
experiment_names = sorted(experiment_names)
experiment_names_for_title = "AND".join(experiment_names)

In [49]:
experiment_names_for_title

'C57_VS_CD1_TRAININGANDC57_vs_CD1_ComparisomANDC57_vs_CD1_ComparisonANDC57_vs_CD1_comparisonANDC57_vs_Cd1_ComparisonANDCD1_VS_CD1_COMPARISON'

- Getting the box numbers

In [50]:
# removing blank spaces
box_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["Box"].unique() if number]
# sorting numbers
box_numbers = sorted(box_numbers)
box_numbers_for_title = "_".join(box_numbers)

In [51]:
box_numbers_for_title

'1_2_3_4'

- Getting the script names

In [52]:
# removing blank spaces
script_names = ["_".join(name.split()) for name in recording_and_metadata_df["MSN"].unique() if name]
# sorting names
script_names = sorted(script_names)
script_names_for_title = "_".join(script_names)

In [53]:
script_names_for_title

'C57_reward_training_CD1_reward_training'

## Adding other useful information

- Adding the date as the number of subsequent days since the start

In [54]:
recording_and_metadata_df["date"].unique()

array(['20220722', '20220723', '20220727', '20220726', '20220719',
       '20220718', '20220720', '20220721', '20220724', '20220725'],
      dtype=object)

In [55]:
for subject in recording_and_metadata_df["subject"].unique():
    print(subject)

1
4
2
3


In [56]:
subject_to_date_to_session = defaultdict(dict)

for subject in recording_and_metadata_df["subject"].unique():
    subject_df = recording_and_metadata_df[recording_and_metadata_df["subject"] == subject]
    for index, date in enumerate(sorted(subject_df["date"].unique())):
        subject_to_date_to_session[subject][date] = index + 1

In [57]:
subject_to_date_to_session

defaultdict(dict,
            {'1': {'20220718': 1,
              '20220719': 2,
              '20220720': 3,
              '20220721': 4,
              '20220722': 5,
              '20220723': 6,
              '20220724': 7,
              '20220725': 8,
              '20220726': 9,
              '20220727': 10},
             '4': {'20220718': 1,
              '20220719': 2,
              '20220720': 3,
              '20220721': 4,
              '20220722': 5,
              '20220723': 6,
              '20220724': 7,
              '20220725': 8,
              '20220726': 9,
              '20220727': 10},
             '2': {'20220718': 1,
              '20220719': 2,
              '20220720': 3,
              '20220721': 4,
              '20220722': 5,
              '20220723': 6,
              '20220724': 7,
              '20220725': 8,
              '20220726': 9,
              '20220727': 10},
             '3': {'20220718': 1,
              '20220719': 2,
              '20220720': 3,

In [58]:
recording_and_metadata_df["session"] = recording_and_metadata_df.apply(lambda x: subject_to_date_to_session[x["subject"]][x["date"]], axis=1)

In [59]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage,session
0,6.22,64.0,399.0,0.0,60.01,6.26,1.0,44.0,36.0,12.0,...,07/22/22,1,C57_vs_CD1_comparison,Cage4,1,12:26:57,13:36:44,CD1_reward_training,4,5
1,6.28,144.0,399.0,0.0,140.01,6.36,1.0,4.0,38.0,12.0,...,07/22/22,1,C57_vs_CD1_comparison,Cage4,1,12:26:57,13:36:44,CD1_reward_training,4,5
2,6.38,234.0,399.0,0.0,230.01,6.41,1.0,34.0,39.0,12.0,...,07/22/22,1,C57_vs_CD1_comparison,Cage4,1,12:26:57,13:36:44,CD1_reward_training,4,5
3,6.44,314.0,399.0,0.0,310.01,6.52,1.0,54.0,40.0,12.0,...,07/22/22,1,C57_vs_CD1_comparison,Cage4,1,12:26:57,13:36:44,CD1_reward_training,4,5
4,6.64,389.0,399.0,0.0,385.01,6.69,1.0,9.0,42.0,12.0,...,07/22/22,1,C57_vs_CD1_comparison,Cage4,1,12:26:57,13:36:44,CD1_reward_training,4,5


In [60]:
recording_and_metadata_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage,session
1957029,3651.72,NaN,NaN,NaN,NaN,3651.74,NaN,NaN,NaN,NaN,...,07/24/22,3,C57_vs_CD1_comparison,Cage 6,1,14:53:31,15:56:03,CD1_reward_training,6,7
1957030,3651.81,NaN,NaN,NaN,NaN,3651.88,NaN,NaN,NaN,NaN,...,07/24/22,3,C57_vs_CD1_comparison,Cage 6,1,14:53:31,15:56:03,CD1_reward_training,6,7
1957031,3655.18,NaN,NaN,NaN,NaN,3655.24,NaN,NaN,NaN,NaN,...,07/24/22,3,C57_vs_CD1_comparison,Cage 6,1,14:53:31,15:56:03,CD1_reward_training,6,7
1957032,3655.31,NaN,NaN,NaN,NaN,3655.36,NaN,NaN,NaN,NaN,...,07/24/22,3,C57_vs_CD1_comparison,Cage 6,1,14:53:31,15:56:03,CD1_reward_training,6,7
1957033,3655.42,NaN,NaN,NaN,NaN,3655.46,NaN,NaN,NaN,NaN,...,07/24/22,3,C57_vs_CD1_comparison,Cage 6,1,14:53:31,15:56:03,CD1_reward_training,6,7


In [61]:
recording_and_metadata_df.groupby(["subject", "date", "session"]).count().head(n=50)

(P)Portentry  (Q)USdelivery  (R)UStime  \
subject date     session                                           
1       20220718 1                1009            139        139   
        20220719 2                1571            150        150   
        20220720 3                3892            244        244   
        20220721 4                8290            240        240   
        20220722 5               13326            240        240   
        20220723 6                9464            240        240   
        20220724 7               10427            240        240   
        20220725 8               10483            215        215   
        20220726 9               12915            240        240   
        20220727 10              10058            240        240   
2       20220718 1                1176            170        170   
        20220719 2                1420            151        151   
        20220720 3                3363            276        276   
        20220721 4                8605            239        239   
        20220722 5                8105            240        240   
        20220723 6                7750            240        240   
        20220724 7               11770            240        240   
        20220725 8               10255            234        234   
        20220726 9               12568            240        240   
        20220727 10               7394            239        239   
3       20220718 1                1572            200        200   
        20220719 2                2173            136        136   
        20220720 3                5512            257        257   
        20220721 4                6111            236        236   
        20220722 5                7769            220        220   
        20220723 6               11448            239        239   
        20220724 7               12749            240        240   
        20220725 8               12075            240        240   
        20220726 9                9168            240        240   
        20220727 10              15526            240        240   
4       20220718 1                1430            196        196   
        20220719 2                2128            178        178   
        20220720 3                3974            226        226   
        20220721 4                7297            240        240   
        20220722 5                7681            240        240   
        20220723 6               11291            240        240   
        20220724 7                7703            240        240   
        20220725 8                9212            240        240   
        20220726 9               29308            201        202   
        20220727 10              15122            237        237   

                          (W)ITIvalues  (S)CSpresentation  (N)Portexit  \
subject date     session                                                 
1       20220718 1               17507                686         1009   
        20220719 2               10004                392         1571   
        20220720 3               17507                686         3892   
        20220721 4               15006                588         8289   
        20220722 5               15006                588        13325   
        20220723 6               15006                588         9464   
        20220724 7               15006                588        10423   
        20220725 8               15006                588        10483   
        20220726 9               15006                588        12913   
        20220727 10              15006                588        10056   
2       20220718 1               17507                686         1176   
        20220719 2               10004                392         1420   
        20220720 3               17507                686         3363   
        20220721 4               15006                588  

## Adding the strain information

In [62]:
cage_to_strain = {"1": "C57", "2": "C57", "3": "C57", "4": "CD1", "5": "CD1", "6":"CD1"}

In [63]:
recording_and_metadata_df["strain"] = recording_and_metadata_df["cage"].map(cage_to_strain)

In [64]:
recording_and_metadata_df["strain"].unique()

array(['CD1', 'C57'], dtype=object)

## Saving the dataframes(spreadsheets to files) with the metadata as part of the name

- Making necessary directories
    - If you want to use any of the other metadata as part of the name, you will have to swap out the variables in the `format()` and change the name of the folder to match your new name. The variable names are the word that's in front of the `=` at the last line of each cell. The `{}` are where the metadata variables will be inserted into the file name. For more information on formatting strings: https://www.w3schools.com/python/ref_string_format.asp
    - You can also just manually rename the files by replacing everything in `""` and removing the `.format()` part
- **NOTE: You may get an error that the file does not exist. If this is the case, it could be the file name is too long(an issue that may happen when using Jupyter Notebooks on Windows)**

In [65]:
output_directory = "./proc/extracted_recording_data_and_metadata/experiment_{}_cage_{}_date_{}_{}".format(experiment_names_for_title, cage_numbers_for_title, earliest_date, latest_date)

In [66]:
output_directory

'./proc/extracted_recording_data_and_metadata/experiment_C57_VS_CD1_TRAININGANDC57_vs_CD1_ComparisomANDC57_vs_CD1_ComparisonANDC57_vs_CD1_comparisonANDC57_vs_Cd1_ComparisonANDCD1_VS_CD1_COMPARISON_cage_1_2_3_4_5_6_date_20220718_20220727'

In [67]:
os.makedirs(output_directory, exist_ok=True)

In [68]:
metadata_df["cohort"] = cohort
concatted_medpc_df["cohort"] = cohort
recording_and_metadata_df["cohort"] = cohort

In [69]:
metadata_df.to_csv(os.path.join(output_directory, "metadata_{}_cage_{}_date_{}_{}.csv".format(cohort, cage_numbers_for_title, earliest_date, latest_date)))
# metadata_df.to_excel(os.path.join(output_directory, "metadata_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

In [70]:
concatted_medpc_df.to_csv(os.path.join(output_directory, "MEDPC_recording_{}_cage_{}_date_{}_{}.csv".format(cohort, cage_numbers_for_title, earliest_date, latest_date)))
# concatted_medpc_df.to_excel(os.path.join(output_directory, "MEDPC_recording_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

In [71]:
recording_and_metadata_df.to_csv(os.path.join(output_directory, "recording_metadata_{}_cage_{}_date_{}_{}.csv".format(cohort, cage_numbers_for_title, earliest_date, latest_date)))
# recording_and_metadata_df.to_excel(os.path.join(output_directory, "recording_and_metadata_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))